<h1><center>Spatial Analysis</center></h1>

# (A) Tweets

In [ ]:
#Connecting to the MongoDB database 'EngTweetsDb' where the tweets are stored
import pymongo
MONGO_HOST = 'mongodb://localhost/EngTweetsDb'
conn = pymongo.MongoClient(MONGO_HOST)["EngTweetsDb"]["en_tweets_col"]
conn.count()

In [ ]:
#Extracting the 'location' information from the stored tweets
location_id = (list(conn.find({},{"user.location":1})))
location_id

In [ ]:
#Creating a list of all the locations 
location_id =(list(conn.find({},{"user.location":1})))
print(len(location_id))
location_only = []
for i in range(len(location_id)):
    location_str = location_id[i]["user"]["location"]
    location_only.append(location_str)
    
print(location_only)

In [ ]:
#Using 'geopy' package to find out the coordinates of all locations stored in 'location_only' list
#These coordinates are then stored in a MongoDB database 'EngTweetsDb' collection 'tweet_coord'
from geopy.geocoders import Nominatim
geolocator = Nominatim()

for place in location_only:
    #print(place)
    location = geolocator.geocode(place, timeout=50)
    try: 
        print(location.address)
        print((location.latitude, location.longitude))
        client = pymongo.MongoClient(MONGO_HOST)
        db = client.EngTweetsDb
        db.tweet_coord.insert({location.address:(location.latitude,location.longitude)})
        
    except Exception as e:
        print(e)

In [ ]:
#Creating a DataFrame of location name, its latitude and longitude
import pandas as pd

latitude_list = []
longitude_list = []
name_list = []
for place in location_only:
    print(place)
    if place == None: 
        print('Found None. Skipping..')
    else:     
        location = geolocator.geocode(place, timeout = 70)
        try: 
            print(location.address)
            print((location.latitude, location.longitude))
            latitude_list.append(location.latitude)
            longitude_list.append(location.longitude)
            name_list.append(location.address)
            DF = pd.DataFrame({'Name' : name_list,
                               'latitude' : latitude_list,
                               'longitude' : longitude_list})
            
        except Exception as e:
            print(e)

In [ ]:
#Printing the DataFrame
DF

In [ ]:
#Creating a sub-DataFrame from 'DF'
coord = DF[['latitude','longitude']]

In [ ]:
#Creating a list of the values of 'coord' DataFrame
coordlist = coord.values.tolist()

In [ ]:
#Using 'Folium' package to plot the locations in a world map
import folium
from folium.plugins import MarkerCluster

#tiles = CartoDB positron, nothing, CartoDB dark_matter

map = folium.Map(location=[40.416775, -3.703790], tiles = "CartoDB dark_matter", zoom_start=2)

marker_cluster = MarkerCluster().add_to(map)

for point in range(len(coordlist)):
    folium.CircleMarker(coordlist[point], popup=folium.Popup(DF['Name'][point],parse_html=True),color='#3186cc',fill_color='#3186cc',fill=True).add_to(marker_cluster)

#The map created has been stored in the local directory as html.     
map.save('tweetMap.html')

#The map can be seen in the Jupyter Notebook by calling the output 'map'
map

# (B) News Articles

In [ ]:
#Connecting to the MongoDB database 'News' where the news articles corresponding to five most-occurring entities are stored
MONGO_HOST1 = 'mongodb://localhost/News'
conn1 = pymongo.MongoClient(MONGO_HOST1)["News"]
db2 = conn1.articles

In [ ]:
#Passing the list of five most frequently occurring entities in tweets
top_five = ['hindu', 'justin', 'bieber', '#bbmas', 'trump']

In [ ]:
#Extracting the 'content' information from the stored news articles
content_id =(list(db2.find({})))

In [ ]:
#Creating a function to extract only the description/body of the news articles
def textprocessor(s):
    start="'body':"
    end="'categories':"
    body=(s.split(start))[1].split(end)[0]
    return body.strip()

In [ ]:
#For creating a location list, only the country data has been extracted and used
country=[]
for i in range(len(content_id)):
    s=content_id[i][list(content_id[i].keys())[1]]
    location=locationprocessor(s)
    if len(location[0]) == 0:
        country.append(None)
        i = i + 1
        print('Record skipped.')
    else:
        country.append(location[0][0]['country'])
print(country)

In [ ]:
#Using 'geopy' package to find out the coordinates of all locations stored in 'country' list
#These coordinates are then stored in a MongoDB database 'News' collection 'news_coord'
from geopy.geocoders import Nominatim
geolocator = Nominatim()

for place in country:
    location = geolocator.geocode(place, timeout=50)
    try: 
        client = pymongo.MongoClient(MONGO_HOST1)
        db3 = client.News
        db3.news_coord.insert({location.address:(location.latitude,location.longitude)})
        
    except Exception as e:
        print(e)

In [ ]:
#Creating a DataFrame of location name, its latitude and longitude
import pandas as pd

latitude_list = []
longitude_list = []
name_list = []
for place in country:
    print(place)
    print('\n')
    if place == None: 
        print('Found None. Skipping..')
    else:     
        location = geolocator.geocode(place, timeout = 70)
        try: 
            #print(location.address)
            #print((location.latitude, location.longitude))
            latitude_list.append(location.latitude)
            longitude_list.append(location.longitude)
            name_list.append(location.address)
            DF = pd.DataFrame({'Name' : name_list,
                               'latitude' : latitude_list,
                               'longitude' : longitude_list})
            #print('Added to DF!')
            #print('*************************************************************************************************************')
        except Exception as e:
            print(e)

In [ ]:
#Printing the DataFrame
DF

In [ ]:
#Creating a sub-DataFrame from 'DF'
coord=DF[['latitude','longitude']]
#To view the first five rows of the DataFrame
coord.head()

In [ ]:
#List of all the values of 'coord' DataFrame
coordlist1 = coord.values.tolist()

In [ ]:
#Using 'Folium' package to plot the the locations in a world map
import folium
from folium.plugins import MarkerCluster

#tiles = CartoDB positron, nothing, CartoDB dark_matter

map1 = folium.Map(location=[40.416775, -3.703790], tiles = "CartoDB dark_matter", zoom_start=2)

marker_cluster1 = MarkerCluster().add_to(map1)

for point in range(len(coordlist1)):
    folium.CircleMarker(coordlist1[point], popup=folium.Popup(DF['Name'][point],parse_html=True),color='#31CCC5',fill_color='#3186cc',fill=True).add_to(marker_cluster1)

#The map created has been stored in the local directory as html.         
map1.save('newsMap.html')

#The map can be seen in the Jupyter Notebook by calling the output 'map'
map1